In [12]:
import requests
import json


def transform_json(url):
    try:
        # Make the GET request
        response = requests.get(url)

        # Check if the request was successful (status code 200)
        response.raise_for_status()

        # Get the JSON data from the response
        data = response.json()
        data['stat'] = {'average_stat': dict(), 'accumulated_stat': dict()}
        
        for match in data["matches"]:
            match['calculated'] = False
        json_string = json.dumps(data, indent=4)

        # 3. Write the string to a file
        with open(f'/Users/jhsu12/Desktop/jones_cup/json/men/{data["name"]}-{data["id"]}.json', 'w') as f:
            f.write(json_string)

        print("Successfully wrote data to settings.json")
        
    except requests.exceptions.HTTPError as errh:
        print(f"Http Error: {errh}")
    except requests.exceptions.ConnectionError as errc:
        print(f"Error Connecting: {errc}")
    except requests.exceptions.Timeout as errt:
        print(f"Timeout Error: {errt}")
    except requests.exceptions.RequestException as err:
        print(f"Oops: Something Else: {err}")

In [13]:
# The URL of the API endpoint
women_team_id = [i for i in range(11374, 11380)]
men_team_id = [i for i in range(11887, 11895)]
for id in men_team_id:
    url = f"https://jonescup.meetagile.com/api/squad-team-squads/{id}"
    transform_json(url)


Successfully wrote data to settings.json
Successfully wrote data to settings.json
Successfully wrote data to settings.json
Successfully wrote data to settings.json
Successfully wrote data to settings.json
Successfully wrote data to settings.json
Successfully wrote data to settings.json
Successfully wrote data to settings.json


In [14]:
def get_completed_match_id(team_id):
    url = f"https://jonescup.meetagile.com/api/squad-team-squads/{team_id}"
    try:
        response = requests.get(url)
        response.raise_for_status()
        data = response.json()
        match_id = []
        for match in data['matches']:
            if match['status'] == "COMPLETED":
                match_id.append(match['id'])
        return match_id
    except requests.exceptions.RequestException as e:
        print(f"Error fetching match ID {match_id}: {e}")
        return None

In [17]:
def create_accumalated_stat(team_id, match_id):
    team_stats = dict()
    for id in match_id:
        url = f"https://jonescup.meetagile.com/api/match-basketball-stats/{id}"
        response = requests.get(url)
        data = response.json()

        squads = data['squads']
        for squad in squads:
            accu_stat = squad['stats']['accumulatedStats']
            # main team
            if squad['squadId'] == team_id:
                
                # FGA, FGM, 3PM, FTA, TO, OREB, score
                team_stats["fieldGoalsAttempted"] =  accu_stat['fieldGoalsAttempted'] + team_stats.get("fieldGoalsAttempted", 0)
                team_stats["fieldGoalsMade"] = accu_stat["fieldGoalsMade"] + team_stats.get("fieldGoalsMade", 0)
                team_stats["threePointersMade"] = accu_stat["threePointersMade"] + team_stats.get("threePointersMade", 0)
                team_stats["freeThrowsAttempted"] = accu_stat["freeThrowsAttempted"] + team_stats.get("freeThrowsAttempted", 0)
                team_stats["turnovers"] = accu_stat["turnovers"] + team_stats.get("turnovers", 0)
                team_stats["squadOffensiveRebounds"] = accu_stat["squadOffensiveRebounds"] + team_stats.get("squadOffensiveRebounds", 0)
                team_stats["squadDefensiveRebounds"] = accu_stat["squadDefensiveRebounds"] + team_stats.get("squadDefensiveRebounds", 0)
                team_stats["score"] = accu_stat["wonScore"] + team_stats.get("score", 0)
                team_stats["teamMinutesPlayed"] = 40 + ( len(data['rounds']) - 4 ) * 5  # 40 minutes for the first 4 rounds, and 5 minutes for overtime
            # opponent team
            else:
                # FGA, FGM, 3PM, FTA, TO, OREB, DREB, score
                team_stats["Op-fieldGoalsAttempted"] =  accu_stat['fieldGoalsAttempted'] + team_stats.get("Op-fieldGoalsAttempted", 0)
                team_stats["Op-fieldGoalsMade"] = accu_stat["fieldGoalsMade"] + team_stats.get("Op-fieldGoalsMade", 0)
                team_stats["Op-threePointersMade"] = accu_stat["threePointersMade"] + team_stats.get("Op-threePointersMade", 0)
                team_stats["Op-freeThrowsAttempted"] = accu_stat["freeThrowsAttempted"] + team_stats.get("Op-freeThrowsAttempted", 0)
                team_stats["Op-turnovers"] = accu_stat["turnovers"] + team_stats.get("Op-turnovers", 0)
                team_stats["Op-squadOffensiveRebounds"] = accu_stat["squadOffensiveRebounds"] + team_stats.get("Op-squadOffensiveRebounds", 0)
                team_stats["Op-squadDefensiveRebounds"] = accu_stat["squadDefensiveRebounds"] + team_stats.get("Op-squadDefensiveRebounds", 0)
                team_stats["Op-score"] = accu_stat["wonScore"] + team_stats.get("Op-score", 0)
    
    return team_stats



                


    
    

In [19]:
def calculate_advanced_stats(data):
    team_stats = dict()
    # main team
    team_stats['Possessions'] = data['fieldGoalsAttempted'] + data['turnovers'] + 0.44 * data['freeThrowsAttempted'] - data['squadOffensiveRebounds']
    team_stats['OffensiveRating'] = data['score'] / data['possessions'] * 100 if data['possessions'] > 0 else 0
    team_stats['Op-possessions'] = data['Op-fieldGoalsAttempted'] + data['Op-turnovers'] + 0.44 * data['Op-freeThrowsAttempted'] - data['Op-squadOffensiveRebounds']
    team_stats['DefensiveRating'] = data['Op-score'] / data['Op-possessions'] * 100 if data['Op-possessions'] > 0 else 0
    team_stats['Pace'] = 40 * ((data['Possessions'] + data['Op-possessions']) / (2*(data['teamMinutesPlayed'] / 5))) if data['teamMinutesPlayed'] > 0 else 0

    # four factors main team
    team_stats['eFG'] = (data['fieldGoalsMade'] + 0.5 * data['threePointersMade']) / data['fieldGoalsAttempted'] if data['fieldGoalsAttempted'] > 0 else 0
    team_stats['TOV'] = data['turnovers'] / (data['possessions'] + data['squadOffensiveRebounds']) if data['possessions'] > 0 else 0
    team_stats['ORP'] = data['squadOffensiveRebounds'] / (data['squadOffensiveRebounds'] + data['Op-squadDefensiveRebounds']) if (data['squadOffensiveRebounds'] + data['Op-squadDefensiveRebounds']) > 0 else 0
    team_stats['FTR'] = data['freeThrowsAttempted'] / data['fieldGoalsAttempted'] if data['fieldGoalsAttempted'] > 0 else 0

    # four factors opponent team
    team_stats['Op-eFG'] = (data['Op-fieldGoalsMade'] + 0.5 * data['Op-threePointersMade']) / data['Op-fieldGoalsAttempted'] if data['Op-fieldGoalsAttempted'] > 0 else 0
    team_stats['Op-TOV'] = data['Op-turnovers'] / (data['Op-possessions'] + data['Op-squadOffensiveRebounds']) if data['Op-possessions'] > 0 else 0
    team_stats['Op-ORP'] = data['Op-squadOffensiveRebounds'] / (data['Op-squadOffensiveRebounds'] + data['squadDefensiveRebounds']) if (data['Op-squadOffensiveRebounds'] + data['squadDefensiveRebounds']) > 0 else 0
    team_stats['Op-FTR'] = data['Op-freeThrowsAttempted'] / data['Op-fieldGoalsAttempted'] if data['Op-fieldGoalsAttempted'] > 0 else 0

    return team_stats

In [18]:
women_team_id = [i for i in range(11374, 11380)]
all_team_stats = dict()
for team_id in women_team_id:
    all_team_stats[team_id] = dict()
    match_id = get_completed_match_id(team_id)
    if len(match_id) == 0:
        print(f"No completed matches for team {team_id}")
        continue
    print(f"Completed match IDs for team {team_id}: {match_id}")

    ## Get accumalated stats for team id
    accumated_stat = create_accumalated_stat(team_id, match_id)

    ## Calculate OFR, DFR, Possession, Pace, Four Factors
    team_stats = calculate_advanced_stats(accumated_stat)
    all_team_stats[team_id] = team_stats
    print(f"Advanced stats for team {team_id}: {team_stats}")

# Write the advanced stats to a JSON file (tmp folder in docker GCP)
with open('/tmp/all_team_stats.json', 'w') as f:
    json.dump(all_team_stats, f, indent=4)



No completed matches for team 11374
No completed matches for team 11375
No completed matches for team 11376
No completed matches for team 11377


KeyboardInterrupt: 